In [1]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd

import scipy.stats as stats
from scipy.stats import bartlett

from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import SelectFromModel
import statsmodels.api as sm

from . import modeling as ml

# 데이터 불러오기
data = pd.read_excel('C:/Users/SAMSUNG/Documents/vscode/2024/ML/프로젝트/data/features_except_GDP.xlsx')

# 데이터 확인
data.head()

ImportError: attempted relative import with no known parent package

In [ ]:
# 등분산성 검정
def bartlett_test(df, col_y, col, p_value = 0.05, H = 1):
    # H가 1인 경우 이분산성 / H가 0인 경우 등분산
    list= []

    for i in col:
        T, p_val = bartlett(df[df[col_y]==1][i], df[df[col_y]==0][i]) 
        list.append([i, p_val])

    list = pd.DataFrame(list, columns = ['변수', 'p_value'])
    
    if H == 1:
        a = list[(list['p_value'] < p_value)][['변수', 'p_value']].sort_values('p_value')
        return a
    else:
        a = list[(list['p_value'] >= p_value)][['변수', 'p_value']].sort_values('p_value')
        return a
    
# T-Test 검정
def t_test(df, col_y, col, col_h0, col_h1, p_value = 0.05):
    list= []
    for i in col:
        
        data1 = df[df[col_y]==1][i] # label=1인 집단의 i 피처 데이터
        data0 = df[df[col_y]==0][i] # label=0인 집단의 i 피처 데이터

        if (col_h0['변수']==i).any():
            # 등분산성 : wald t-test
            t_stat, p_val = stats.ttest_ind(
                data1, data0, equal_var=True
            ) 
            list.append([i, 'homo', p_val])
        elif (col_h1['변수']==i).any():
            # 이분산성 : welch’s t-test
            t_stat, p_val = stats.ttest_ind(
                data1, data0, equal_var=False
            )
            list.append([i, 'hetero', p_val])

    list = pd.DataFrame(list, columns = ['변수', '분산', 'p_value'])

    # pvalue < 0.05인 피처 데이터만 return
    a = list[(list['p_value'] < p_value)][['변수', '분산', 'p_value']].sort_values('p_value')
    return a

# 피처 셀렉션 : Filter (t-test)
def selection_ttest(train, col_y = 'label', cols_feature = None, pvalue=0.05):
    if cols_feature is None:
        cols_feature = train.columns.difference([col_y])

    """ (1) 등분산성 확인 """

    ## 이분산성 변수
    x_hetero = bartlett_test(train, col_y, cols_feature, H = 1)

    ## 등분산성 변수
    x_homo = bartlett_test(train, col_y, cols_feature, H = 0)
    
    """ (2) T-test 적용 """

    # 2) t_test 결과 p_value < 0.05보다 작은 유의한 변수 가져오기
    x_ttest = t_test(train, col_y, cols_feature, x_homo, x_hetero, p_value=pvalue)

    print("후보 피처 수 : ", len(cols_feature)-1)
    print("유의한 피쳐 수 :", len(x_ttest))

    return x_ttest.sort_values(by="p_value", ascending=True)